In [1]:
import numpy as np
import pandas as pd

from pymol import cmd

from pymol_utils import *
import biotite_helper
from biotite_helper import rcsb, gettempdir, struc, strucio, fasta, seq, align
from biotite_helper import visualize_secondary_structure, plt, SeqIO

import glob, os


In [2]:


#cmd.create("ref")
ensemble_structures = glob.glob("refine_*.sa.pdb_A.pdb")
ref = cmd.load("1irj.cif","ref")
for s in ensemble_structures:
    pdb = cmd.load(s,"to_align")
    al = cmd.align("to_align and (resid 6-85) and name CA",
                   "ref and chain A and (resid 6-85) and name CA")
    print(al)
              


 ExecutiveLoad-Detail: Detected mmCIF
(1.7483736276626587, 74, 5, 2.053497076034546, 80, 404.0, 80)
(2.565891742706299, 80, 1, 2.565891742706299, 80, 404.0, 80)
(3.2104105949401855, 80, 1, 3.2104105949401855, 80, 404.0, 80)
(3.6282758712768555, 80, 1, 3.6282758712768555, 80, 404.0, 80)
(3.964090585708618, 80, 1, 3.964090585708618, 80, 404.0, 80)
(4.241611480712891, 80, 1, 4.241611480712891, 80, 404.0, 80)
(4.45974063873291, 80, 1, 4.45974063873291, 80, 404.0, 80)
(4.606429100036621, 80, 1, 4.606429100036621, 80, 404.0, 80)
(4.769803047180176, 80, 1, 4.769803047180176, 80, 404.0, 80)
(4.890521049499512, 80, 1, 4.890521049499512, 80, 404.0, 80)


In [ ]:
def helix_angle_calculator(pdb_file, helix1=-2, helix2=-1, chain_id="B"):

    # Get title from pdb file
    pdb_title = []
    with open(pdb_file) as f:

        pdb_title = []
        for line in f:
            if line.startswith("TITLE"):
                pdb_title.append(line[6:].strip())

    if len(pdb_title) > 0:
        pdb_title = " ".join(pdb_title)
    else:
        pdb_title = None
    
    # Get pdb id
    pdb_id = os.path.split(pdb_file)[-1].split(".")[0].split("_")[0]
    
    # Load pdb file and
    pdb = cmd.load(pdb_file)
    name = get_pymol_name(pdb_file)
    array = strucio.load_structure(pdb_file)
        
    # Get secondary structures from pdb file
    sse = struc.annotate_sse(array, chain_id="B")
    ss = "".join(sse)

    # Find helix starts and ends
    starts = []
    ends = []
    
    # If first element is helix start there
    if sse[0] == "a":
        starts.append("a")
        
    # Go over rest of sequence
    for i in range(1,len(sse)):
        if sse[i] == "a" and sse[i-1] != "a":
            starts.append(i)
        elif sse[i] != "a" and sse[i-1] == "a":
            ends.append(i)
    
    # If last element is helix, end there
    if sse[-1] == "a":
        ends.append(i)
    
    # List of tuples containing each helix (start,end)
    helices = list(zip(starts,ends))
        
    # Get specified helix spans
    h1 = helices[helix1]
    h2 = helices[helix2]
    
    # Select by pymol
    h1_sele = f"/{name}//{chain_id}/{h1[0]}-{h1[1]}/"
    h2_sele = f"/{name}//{chain_id}/{h2[0]}-{h2[1]}/"
        
    # HACK. Secondary structure caller messing up on this structure and getting
    # wrong helices
    if pdb_id == "1irj":
        h1 = (56,66)
        h2 = (76,86)
        h1_sele = f"/{name}//B/56-66/"
        h2_sele = f"/{name}//B/76-86/"

    # Get angles using angle_between
    angle = angle_between_helices(h1_sele, h2_sele,
                                  method="cafit_orientation",quiet=True)
    
    return pdb_title, pdb_id, angle, h1, h2

### Calculate angles across published S100 structures

In [ ]:
apo = glob.glob("../apo/*/*.pdb")
ca = glob.glob("../ca/*/*.pdb")
remap_names = {"S100C":"S100A11"}

files = apo + ca

pdb_titles = []
pdb_ids = []
angles = []
h3s = []
h4s = []

calcium_state = []
target_state = []

for pdb_file in files:
    
    if "apo" in pdb_file:
        calcium_state.append(False)
    if "ca" in pdb_file:
        calcium_state.append(True)
        
    if "no-peptide" in pdb_file:
        target_state.append(False)
    else:
        target_state.append(True)
        
    pdb_title, pdb_id, angle, h3, h4 = helix_angle_calculator(pdb_file)
    
    s100_name = re.search("s100.*?($|[;, \)])",pdb_title,re.IGNORECASE)
    if s100_name is not None:
        start, end = s100_name.span()
        s100_name = pdb_title[start:end]
        s100_name = re.sub("[;, \)]","",s100_name)
    else:
        if re.search("MRP14",pdb_title):
            s100_name = "S100A9"
        elif re.search("MIGRATION INHIBITORY FACTOR-RELATED PROTEIN 8",pdb_title):
            s100_name = "S100A8"
        else:
            s100_name = None
            
    try:
        s100_name = remap_names[s100_name]
    except KeyError:
        pass

    pdb_ids.append(pdb_id)
    pdb_titles.append(s100_name)
    angles.append(180 - angle)
    h3s.append(h3)
    h4s.append(h4)
    
        
df = pd.DataFrame({"pdb_id":pdb_ids, "S100":pdb_titles, "calcium_bound":calcium_state, "target_bound":target_state,"h3_span":h3s, "h4_span":h4s, "h3_h4 angle":angles})

apo_angles = df.loc[np.logical_not(df.calcium_bound),"h3_h4 angle"]
print(f"Apo H3/H4 angles: {np.mean(apo_angles):.3f} +/- {np.std(apo_angles):.2f}")

apo_angles = df.loc[df.calcium_bound,"h3_h4 angle"]
print(f"Ca-bound H3/H4 angles: {np.mean(apo_angles):.3f} +/- {np.std(apo_angles):.2f}")

angle = float(df.loc[df.pdb_id == "1irj","h3_h4 angle"].iloc[0])
print(f"Ca-bound S100A9 H3/H4 angle: {angle:.3f}")

df.to_csv("table-s2_s100-h3-h4-angles.csv")
df

### Calculate angles across hS100A9/M63F NMR ensemble

In [ ]:

angles = []
for pdb_file in glob.glob("../nmr-structures/*B.pdb"):
    
    pdb = cmd.load(pdb_file)
    name = get_pymol_name(pdb_file)
    
    h1_sele = f"/{name}/B//56-66/"
    h2_sele = f"/{name}/B//76-86/"
    
    angle = angle_between_helices(h1_sele, h2_sele,
                                  method="cafit_orientation",quiet=True)

    angles.append(180-angle)
    
print(f"M63F H3/H4 angle: {np.mean(angles):.3f} +/- {np.std(angles):.2f}")
    